# Method 1 - Based on Ranking Position

In [ ]:
df = pd.read_csv('semrush file.csv')
df.head(3)

In [ ]:
index_list=[]
for kw in df['Keyword'].unique():
    if df['Keyword'].tolist().count(kw)==1:
        index = df[df['Keyword']==kw].index[0]
        index_list.append(index)
    else:
        index = df[df['Keyword']==kw].sort_values(by='Position').index[0]
        index_list.append(index)

result_df = df[df.index.isin(index_list)]

# Method 2 - Based on Keyword Ownership Score

In [ ]:
df = pd.read_csv('Dataset for Keyword Ownership.csv')
newdf = df[['Keyword','URL','Position','Traffic','PA']]
newdf.head()

,Keyword,URL,Position,Traffic,PA
0,url profiler,https://ipullrank.com/how-to-run-screaming-fro...,25,0,39
1,url profiler,https://ipullrank.com/9-problems-with-content-...,26,0,26
2,king agency,https://ipullrank.com/,71,0,47
3,king agency,https://ipullrank.com/about-us/,72,0,40
4,machine learning chat bots,https://ipullrank.com/machine-learning-guide/h...,20,0,26


Step 1 : Generate expectations

In [ ]:
for kw in newdf['Keyword'].unique():

    subdf = newdf[newdf['Keyword']==kw]

    for col in ['Position','Traffic','PA']: # used to be Search Vol, Keyword Difficulty, PA
        
        mean=subdf[col].mean()
        if mean != 0:
          values=[]
          if col == 'Position':
              for i in subdf.index:
                  values.append(1-subdf[col][i]/mean)
          else:
              for i in subdf.index:
                  values.append(subdf[col][i]/mean)
          
          index = subdf.index.tolist()
          newdf.loc[index, '% of Avg '+col] = values
        else:
          newdf['% of Avg '+col] = [0]*len(newdf)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [ ]:
newdf.head()

,Keyword,URL,Position,Traffic,PA,% of Avg Position,% of Avg Traffic,% of Avg PA
0,url profiler,https://ipullrank.com/how-to-run-screaming-fro...,25,0,39,0.019608,0,1.20000
1,url profiler,https://ipullrank.com/9-problems-with-content-...,26,0,26,-0.019608,0,0.80000
2,king agency,https://ipullrank.com/,71,0,47,0.006993,0,1.08046
3,king agency,https://ipullrank.com/about-us/,72,0,40,-0.006993,0,0.91954
4,machine learning chat bots,https://ipullrank.com/machine-learning-guide/h...,20,0,26,0.024390,0,1.00000


Step 2: Assign weight to each category and generate weighted values

In [ ]:
weighted = []
for i in newdf.index:
    value = newdf['% of Avg Position'][i]*0.3 +\
            newdf['% of Avg Traffic'][i]*0.4 +\
            newdf['% of Avg PA'][i]*0.3
    weighted.append(value)
newdf['Weighted Value'] = weighted

Step 3: Generate the Score

In [ ]:
newdf.loc[:,'Keyword Ownership Score'] = 27*np.log(newdf['Weighted Value'])+100
newdf.loc[:,'Keyword Ownership Score'] = newdf['Keyword Ownership Score'].round(0)
newdf.head(6)

,Keyword,URL,Position,Traffic,PA,% of Avg Position,% of Avg Traffic,% of Avg PA,Weighted Value,Keyword Ownership Score
0,url profiler,https://ipullrank.com/how-to-run-screaming-fro...,25,0,39,0.019608,0,1.20000,0.365882,73.0
1,url profiler,https://ipullrank.com/9-problems-with-content-...,26,0,26,-0.019608,0,0.80000,0.234118,61.0
2,king agency,https://ipullrank.com/,71,0,47,0.006993,0,1.08046,0.326236,70.0
3,king agency,https://ipullrank.com/about-us/,72,0,40,-0.006993,0,0.91954,0.273764,65.0
4,machine learning chat bots,https://ipullrank.com/machine-learning-guide/h...,20,0,26,0.024390,0,1.00000,0.307317,68.0
5,machine learning chat bots,https://ipullrank.com/machine-learning-guide/c...,21,0,26,-0.024390,0,1.00000,0.292683,67.0
